In [1]:
import numpy as np
import math
import copy

In [3]:
from sklearn import model_selection, datasets, svm
from sklearn.preprocessing import normalize

In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

In [34]:
iris = datasets.load_iris()
X = iris.data
# X = normalize(iris.data)
Y = iris.target
X_train, X_test, Y_train, Y_text = model_selection.train_test_split(X_norm, Y, test_size=0.33, random_state=42)

In [7]:
N = 4

In [8]:
def feature_map(X):
    q = QuantumRegister(N, name='q')
    c = ClassicalRegister(2, name='c1')
    qc = QuantumCircuit(q, c)
    
    for i, x in enumerate(X_train[0]):
        qc.rx(x, i) 
        
    return qc, c

In [9]:
def variational_circuit(qc, theta):
    for i in range(N-1):
        qc.cx(i, i+1)

    qc.cx(N-1, 0)
    qc.barrier()
    for i in range(N):
        qc.ry(theta[i], i)

    qc.barrier()
    
    return qc

In [10]:
def binary2int(bstring: str):
    if not isinstance(bstring, str):
        bstring = str(bstring)
    return int(bstring, 2)

# print(binary2int("101101"))

In [11]:
def get_max(dict_to_verify: dict):
    return max(dict_to_verify, key=dict_to_verify.get)

In [12]:
def quantum_nn(X, theta, simulator=True):
    qc, c = feature_map(X)
    
    qc.barrier()
    qc = variational_circuit(qc, theta)
    qc.measure(0,c[0])
    qc.measure(1,c[1])
    
    # parte que deve ser alterada para a nova versão do qiskit
    shots = 1E4
    # backend = Aer.get_backend("qasm_simulator")
    results = AerSimulator().run(qc, shots=1E4, memory=True).result()
    # job = qiskit.execute(qc, backend, shots=shots)
    # result = job.result()
    counts = results.get_counts(qc)

    # até aqui precisa ser refatorado para nova versão
    
    # print(counts)
    # print(counts['1']/shots)

    # aqui que precisa ser montado o esquema de verificação de encontro para os valores dos dados necessários
    stats = {}
    for key, value in counts.items():
        stats[binary2int(key)] = value / shots

    result = get_max(stats)
    # return stats
    return result

In [13]:
def loss(prediction, target):
    return math.pow(target-prediction, 2)

In [22]:
def gradient(X, Y, theta):
    delta = 0.01
    grad = []
    for i in range(len(theta)):
        dtheta = copy.copy(theta)
        dtheta[i] += delta

        pred1 = quantum_nn(X, dtheta)
        pred2 = quantum_nn(X, theta)

        grad.append((loss(pred1, Y) - loss(pred2,Y))/delta)

    return np.array(grad) 

In [15]:
prediction = quantum_nn(X_train[5], np.random.rand(N))
target = Y_train[5]
print(prediction, target)

0 0


In [16]:
loss(prediction, target)

0.0

In [ ]:
eta = 0.05
loss_list = []
theta = np.random.rand(N)

for _ in range(20):
    
    loss_tmp = []
    for X_i, Y_i in zip(X_train, Y_train):
        prediction = quantum_nn(X_i, theta)
        loss_tmp.append(loss(prediction, Y_i))
        theta = theta - eta * gradient(X_i, Y_i, theta) # gradiente descendent

    print(f"Epoch: {_+1:2}\tMean: {np.mean(loss_tmp)}")
    loss_list.append(np.mean(loss_tmp))

Epoch:  1	1.71
Epoch:  2	1.71
Epoch:  3	1.71
Epoch:  4	1.71
Epoch:  5	1.71
Epoch:  6	1.71
Epoch:  7	1.71
Epoch:  8	1.71
Epoch:  9	1.71
Epoch: 10	1.71
Epoch: 11	1.71
Epoch: 12	1.71
Epoch: 13	1.71
Epoch: 14	1.71
Epoch: 15	1.71
Epoch: 16	1.71
Epoch: 17	1.71
Epoch: 18	1.71
Epoch: 19	1.71


In [36]:
loss_list

[1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71,
 1.71]